In [20]:
import pandas as pd
import numpy as np
from osgeo import gdal, osr, ogr
import sys
import math
import os

In [3]:
SPATIAL_IDX = 5
TEMPORAL_IDX = 9

In [4]:
'''
Open input file <filename> and return numpy array
'''

def getModisData(filename):
    # This allows gdal to throw python exceptions
    gdal.UseExceptions()
    try:
        modisRaster = gdal.Open(filename)
    except e:
        print("Unable to open tif file")
        print(e)
        print(modisRaster.GetMetadata())
    modisData = modisRaster.ReadAsArray()
    # set NaN values to 0
    modisData = np.where((modisData < 1) | (modisData > 366), 0, modisData)
    modisData = np.floor(modisData)
    return modisData,modisRaster
    

In [5]:
'''
Include any pixels that fall within temporal index within window to current event ID
'''
def setEventPixels(inputWindow,eventWindow,Id,burnDay):
    #print("input window ID={}, burnDay={}:".format(Id,burnDay))
    #print(inputWindow)
    mask = np.logical_and(inputWindow>0,abs(inputWindow-burnDay)<=TEMPORAL_IDX)
    overlap = np.logical_and(eventWindow>0,mask)
    if overlap.any() :
        #print("Overlapping pixels")
        Id = np.amin(eventWindow[overlap],axis=None)
    eventWindow[mask] = Id
    #print("Event window:")
    #print(eventWindow)

In [6]:
'''
Process the input array determining fire events based on SPATIAL_IDX and TEMPORAL_IDX
'''
def getEventData(inputArray):
    currEventId = 0
    eventArray = np.zeros(inputArray.shape)
    #Traverse pixels in the MODIS Data Array to look for neighboring event pixels that belong to the event
    for row in range(inputArray.shape[0]):
        for col in range(inputArray.shape[1]):
            if inputArray[row,col] : # burn detected
                #print("{},{} burn detected".format(row,col))
                # set spatial window
                top = max(0,row-SPATIAL_IDX)
                bottom = min(inputArray.shape[0],row+SPATIAL_IDX+1)
                left = max(0,col-SPATIAL_IDX)
                right = min(inputArray.shape[1],col+SPATIAL_IDX+1)
                id = eventArray[row,col]
                if id == 0 : #Event Id not set so get next ID
                    currEventId += 1
                    id = currEventId
                setEventPixels(inputArray[top:bottom,left:right],eventArray[top:bottom,left:right],id,inputArray[row,col])
    #print(eventArray)
    return eventArray

In [7]:
'''
Save the event data to GTiff file
'''
def saveEventData(eventData,input_raster,outfile):
    rows = input_raster.RasterYSize
    cols = input_raster.RasterXSize
    output_raster = gdal.GetDriverByName('GTiff').Create(outfile, cols, rows, 1 ,gdal.GDT_Float32)
    output_raster.SetGeoTransform(input_raster.GetGeoTransform())
    output_raster.SetProjection(input_raster.GetProjection())
    output_raster.GetRasterBand(1).WriteArray(eventData) 

In [22]:
input_path = '../data/MCD64A1/C6/yearly_events/'
out_path = '../data/MCD64A1/C6/yearly_events/'

for i in range(2001,2016) :
    if not os.path.exists(input_path + 'USA_burnevents_' + str(i) + '.tif'):
        modisData,modisRaster = getModisData(out_path + 'USA_BurnDate_' + str(i) + '.tif')
        eventData = getEventData(modisData)
        saveEventData(eventData,modisRaster, out_path + 'USA_burnevents_' + str(i) + '.tif')

Just a quick check to make sure the saved data matches the event data. You can delete.

In [37]:
savedRaster = gdal.Open('../out/WUS_BurnDate_2014_out.tif')
savedData = savedRaster.ReadAsArray()
print(np.count_nonzero(eventData))
print(np.count_nonzero(savedData))
print(np.count_nonzero(eventData-savedData))

48060
48060
0
